In [1]:
from scipy.stats import norm
from numpy import mean
from numpy import std
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
sns.set()

In [2]:
def fit_distribution(data):
    mu = mean(data)
    sigma = std(data)
    print(mu, sigma)
    dist = norm(mu, sigma)
    return dist

In [3]:
sk_lt=[]
for i in range(200,6019):
    sk_lt.append(i)

In [4]:
s=pd.read_csv("C:\\Users\\vigne\\Downloads\\train-data.csv",skiprows=sk_lt)

In [5]:
s.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74


In [6]:
s['Transmission'].unique()

array(['Manual', 'Automatic'], dtype=object)

In [7]:
for i in range(0,len(s[['Power']])):
    if pd.isna(i)==True:
        s['Power'][i]='0'
    elif s['Power'][i]=='null bhp':
        s['Power'][i]='0'


<ipython-input-7-16914dd481f1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s['Power'][i]='0'


In [8]:
s=s.fillna(0)

In [9]:
df=s[['Kilometers_Driven','Transmission']]
df1= s[['Mileage','Price']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
df2=pd.concat([df1,df],axis=1, join='inner')
df2['Mileage'] = df2['Mileage'].astype(float)
df2['Price'] =df2['Price'].astype(float)
df2

,Mileage,Price,Kilometers_Driven,Transmission
0,26.60,1.75,72000,Manual
1,19.67,12.50,41000,Manual
2,18.20,4.50,46000,Manual
3,20.77,6.00,87000,Manual
4,15.20,17.74,40670,Automatic
...,...,...,...,...
195,25.80,3.50,52000,Manual
196,20.36,3.55,43571,Manual
197,25.80,3.25,50000,Manual
198,17.50,4.50,113000,Manual


In [10]:
df=df2[df2['Transmission']=='Manual']
df.shape

(137, 4)

In [11]:
df1=df2[df2['Transmission']=='Automatic']
df1.shape

(63, 4)

In [12]:
y=df2['Transmission'].map({'Manual':0,'Automatic': 1})
y

0      0
1      0
2      0
3      0
4      1
      ..
195    0
196    0
197    0
198    0
199    0
Name: Transmission, Length: 200, dtype: int64

In [13]:
df.drop(['Transmission'],axis=1)
df1.drop(['Transmission'],axis=1)
df2.drop(['Transmission'],axis=1)

,Mileage,Price,Kilometers_Driven
0,26.60,1.75,72000
1,19.67,12.50,41000
2,18.20,4.50,46000
3,20.77,6.00,87000
4,15.20,17.74,40670
...,...,...,...
195,25.80,3.50,52000
196,20.36,3.55,43571
197,25.80,3.25,50000
198,17.50,4.50,113000


In [14]:
priory0 = df.shape[0] / df2.shape[0]

In [15]:
priory1 = df1.shape[0] / df2.shape[0]

In [16]:
print(priory0,priory1)

0.685 0.315


In [17]:
distX1y0 = fit_distribution(df.iloc[:, 0])
distX2y0 = fit_distribution(df.iloc[:, 1])
distX3y0 = fit_distribution(df.iloc[:, 2])

distX1y1 = fit_distribution(df1.iloc[:, 0])
distX2y1 = fit_distribution(df1.iloc[:, 1])
distX3y1 = fit_distribution(df1.iloc[:, 2])

19.330583941605845 4.706094009568382
5.198978102189781 3.244283076174551
63548.131386861314 38838.67123307157
15.702857142857143 4.648663719437795
18.776190476190482 12.502596483165473
49035.206349206346 25732.40094607349


In [18]:
xsamp=df2.iloc[0:1,:3].values.tolist()
xsamp

[[26.6, 1.75, 72000.0]]

In [19]:
ysamp=df2.iloc[0:1,3:4].values.tolist()
ysamp

[['Manual']]

In [20]:
X= [item for sublist in xsamp for item in sublist]
Y = [item for sublist in ysamp for item in sublist]
print(X,Y)

[26.6, 1.75, 72000.0] ['Manual']


In [21]:
py0=priory0 * distX1y0.pdf(X[0]) * distX2y0.pdf(X[1]) * distX3y0.pdf(X[2])
py0

1.2346917761931136e-08

In [22]:
py1=priory0 * distX1y1.pdf(X[0]) * distX2y1.pdf(X[1]) * distX3y1.pdf(X[2])
py1

4.951404085837913e-10

In [23]:
py0>py1

True

In [24]:
y

0      0
1      0
2      0
3      0
4      1
      ..
195    0
196    0
197    0
198    0
199    0
Name: Transmission, Length: 200, dtype: int64

In [25]:
df2.drop(['Transmission'],axis=1,inplace=True)

In [26]:
from sklearn.datasets import make_blobs
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(df2,y)

GaussianNB()

In [30]:
model.predict(xsamp)

array([0], dtype=int64)

In [35]:
df=pd.read_csv("C:\\Users\\vigne\\Downloads\\train-data.csv")

In [36]:
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74
